In [1]:
import pandas as pd
import gmaps
import requests
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Otane,NZ,-39.88,176.63,55.99,84,100,2.35,overcast clouds,0.00,0
1,1,Qaanaaq,GL,77.48,-69.36,34.07,84,100,2.82,overcast clouds,0.00,0
2,2,Lebu,CL,-37.62,-73.65,52.66,86,100,16.49,light rain,0.21,0
3,3,Victoria,HK,22.29,114.16,93.00,66,26,15.01,scattered clouds,0.00,0
4,4,Cherskiy,RU,68.75,161.30,83.64,23,3,5.26,clear sky,0.00,0


In [3]:
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
Rain (inches)          float64
Snow (inches)            int64
dtype: object

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# Ask the customer about rain and snow
rain_amount = input("Do you want it to be raining? (yes/no)")
snow_amount = input("Do you want it to be snowing? (yes/no)")

Do you want it to be raining? (yes/no)no
Do you want it to be snowing? (yes/no)no


In [7]:
if rain_amount == "no" and snow_amount == "no":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                          (city_data_df["Max Temp"] >= min_temp) &
                                          (city_data_df["Rain (inches)"] == 0) & 
                                          (city_data_df["Snow (inches)"] == 0)]
elif rain_amount == "no" and snow_amount =="yes":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                          (city_data_df["Max Temp"] >= min_temp) &
                                          (city_data_df["Rain (inches)"] == 0) & 
                                          (city_data_df["Snow (inches)"] >= 0.0)]
elif rain_amount == "yes" and snow_amount =="no":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                          (city_data_df["Max Temp"] >= min_temp) &
                                          (city_data_df["Rain (inches)"] > 0.0) & 
                                          (city_data_df["Snow (inches)"] == 0)]
else:
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                          (city_data_df["Max Temp"] >= min_temp) &
                                          (city_data_df["Rain (inches)"] > 0.0) & 
                                          (city_data_df["Snow (inches)"] > 0)]

filtered_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
4,4,Cherskiy,RU,68.75,161.30,83.64,23,3,5.26,clear sky,0.0,0
9,9,Hilo,US,19.73,-155.09,80.60,61,90,5.82,overcast clouds,0.0,0
10,10,Wahiawa,US,21.50,-158.02,82.40,57,90,13.87,overcast clouds,0.0,0
11,11,Atuona,PF,-9.80,-139.03,80.29,74,88,14.61,overcast clouds,0.0,0
12,12,Beyneu,KZ,45.32,55.20,81.41,29,5,21.50,clear sky,0.0,0


In [8]:
filtered_cities_df.count()

City_ID                236
City                   236
Country                236
Lat                    236
Lng                    236
Max Temp               236
Humidity               236
Cloudiness             236
Wind Speed             236
Current Description    236
Rain (inches)          236
Snow (inches)          236
dtype: int64

In [57]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = filtered_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Cherskiy,RU,83.64,clear sky,68.75,161.30,
9,Hilo,US,80.60,overcast clouds,19.73,-155.09,
10,Wahiawa,US,82.40,overcast clouds,21.50,-158.02,
11,Atuona,PF,80.29,overcast clouds,-9.80,-139.03,
12,Beyneu,KZ,81.41,clear sky,45.32,55.20,
14,Mandalgovi,MN,77.54,overcast clouds,45.76,106.27,
17,Butaritari,KI,80.37,overcast clouds,3.07,172.79,
21,Rikitea,PF,75.29,overcast clouds,-23.12,-134.97,
24,Saskylakh,RU,82.44,broken clouds,71.92,114.08,
25,Umm Lajj,SA,80.62,clear sky,25.02,37.27,


In [58]:
# Dependencies and Setup
import requests
import gmaps
import pprint

# Import API key
from config import g_key

# Set the parameters to search for a hotel.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    #hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [59]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Cherskiy,RU,83.64,clear sky,68.75,161.30,Gostinitsa
9,Hilo,US,80.60,overcast clouds,19.73,-155.09,Hilo Hawaiian Hotel
10,Wahiawa,US,82.40,overcast clouds,21.50,-158.02,Aloha Wahiawa
11,Atuona,PF,80.29,overcast clouds,-9.80,-139.03,Villa Enata
12,Beyneu,KZ,81.41,clear sky,45.32,55.20,NUR


In [60]:
hotel_df.count()

City                   236
Country                236
Max Temp               236
Current Description    236
Lat                    236
Lng                    236
Hotel Name             236
dtype: int64

In [61]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [62]:
vacation_df =pd.read_csv("weather_data/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,4,Cherskiy,RU,83.64,clear sky,68.75,161.30,Gostinitsa
1,9,Hilo,US,80.60,overcast clouds,19.73,-155.09,Hilo Hawaiian Hotel
2,10,Wahiawa,US,82.40,overcast clouds,21.50,-158.02,Aloha Wahiawa
3,11,Atuona,PF,80.29,overcast clouds,-9.80,-139.03,Villa Enata
4,12,Beyneu,KZ,81.41,clear sky,45.32,55.20,NUR


In [65]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [68]:
import matplotlib
import matplotlib.pyplot as plt
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>